### **Mask R-CNN video inference**

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
!mkdir ./pretrained
!wget -O ./pretrained/mask_rcnn_inception_v2_coco_2018_01_28.tar.gz http://download.tensorflow.org/models/object_detection/mask_rcnn_inception_v2_coco_2018_01_28.tar.gz
!wget -O ./pretrained/config_mask_graph.pbtxt https://raw.githubusercontent.com/opencv/opencv_extra/master/testdata/dnn/mask_rcnn_inception_v2_coco_2018_01_28.pbtxt

--2022-08-12 14:48:59--  http://download.tensorflow.org/models/object_detection/mask_rcnn_inception_v2_coco_2018_01_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.45.112, 2607:f8b0:4004:83f::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.45.112|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 177817887 (170M) [application/x-tar]
Saving to: ‘./pretrained/mask_rcnn_inception_v2_coco_2018_01_28.tar.gz’

./pretrained/mask_r 100%[===================>] 169.58M   315MB/s    in 0.5s    

2022-08-12 14:48:59 (315 MB/s) - ‘./pretrained/mask_rcnn_inception_v2_coco_2018_01_28.tar.gz’ saved [177817887/177817887]

--2022-08-12 14:48:59--  https://raw.githubusercontent.com/opencv/opencv_extra/master/testdata/dnn/mask_rcnn_inception_v2_coco_2018_01_28.pbtxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.co

In [ ]:
!tar -xvf ./pretrained/mask_rcnn_inception_v2_coco_2018_01_28.tar.gz -C ./pretrained

mask_rcnn_inception_v2_coco_2018_01_28/
mask_rcnn_inception_v2_coco_2018_01_28/model.ckpt.index
mask_rcnn_inception_v2_coco_2018_01_28/checkpoint
mask_rcnn_inception_v2_coco_2018_01_28/pipeline.config
mask_rcnn_inception_v2_coco_2018_01_28/model.ckpt.data-00000-of-00001
mask_rcnn_inception_v2_coco_2018_01_28/model.ckpt.meta
mask_rcnn_inception_v2_coco_2018_01_28/saved_model/
mask_rcnn_inception_v2_coco_2018_01_28/saved_model/saved_model.pb
mask_rcnn_inception_v2_coco_2018_01_28/saved_model/variables/
mask_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb


In [ ]:
labels_to_names_seq = {0:'person',1:'bicycle',2:'car',3:'motorcycle',4:'airplane',5:'bus',6:'train',7:'truck',8:'boat',9:'traffic light',
                       10:'fire hydrant',11:'street sign',12:'stop sign',13:'parking meter',14:'bench',15:'bird',16:'cat',17:'dog',18:'horse',19:'sheep',
                       20:'cow',21:'elephant',22:'bear',23:'zebra',24:'giraffe',25:'hat',26:'backpack',27:'umbrella',28:'shoe',29:'eye glasses',
                       30:'handbag',31:'tie',32:'suitcase',33:'frisbee',34:'skis',35:'snowboard',36:'sports ball',37:'kite',38:'baseball bat',39:'baseball glove',
                       40:'skateboard',41:'surfboard',42:'tennis racket',43:'bottle',44:'plate',45:'wine glass',46:'cup',47:'fork',48:'knife',49:'spoon',
                       50:'bowl',51:'banana',52:'apple',53:'sandwich',54:'orange',55:'broccoli',56:'carrot',57:'hot dog',58:'pizza',59:'donut',
                       60:'cake',61:'chair',62:'couch',63:'potted plant',64:'bed',65:'mirror',66:'dining table',67:'window',68:'desk',69:'toilet',
                       70:'door',71:'tv',72:'laptop',73:'mouse',74:'remote',75:'keyboard',76:'cell phone',77:'microwave',78:'oven',79:'toaster',
                       80:'sink',81:'refrigerator',82:'blender',83:'book',84:'clock',85:'vase',86:'scissors',87:'teddy bear',88:'hair drier',89:'toothbrush',
                       90:'hair brush'}

In [ ]:
colors = list(
    [[0, 255, 0],
     [0, 0, 255],
     [255, 0, 0],
     [0, 255, 255],
     [255, 255, 0],
     [255, 0, 255],
     [80, 70, 180],
     [250, 80, 190],
     [245, 145, 50],
     [70, 150, 250],
     [50, 190, 190]])

In [ ]:
def get_box_info(box, image_width, image_height):
  class_id = int(box[1])
  x1 = int(image_width * box[3])
  y1 = int(image_height * box[4])
  x2 = int(image_width * box[5])
  y2 = int(image_height * box[6])

  x1 = max(0, min(x1, image_width-1))
  y1 = max(0, min(y1, image_height-1))
  x2 = max(0, min(x2, image_width-1))
  y2 = max(0, min(y2, image_height-1))
  return class_id, x1, y1, x2, y2

def draw_box(draw_image, box, image_width, image_height):
  green_color=(0, 255, 0)
  red_color=(0, 0, 255)
  confidence_score = box[2]    
  class_id, x1, y1, x2, y2 = get_box_info(box, image_width, image_height)
  text = '{}: {:.4f}'.format(labels_to_names_seq[class_id], confidence_score)
  cv2.rectangle(draw_image, (x1, y1), (x2, y2), green_color, thickness=2)
  cv2.putText(draw_image, text, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, red_color, 1)
  return draw_image

def draw_mask(draw_image, box, mask, image_width, image_height, mask_threshold):
  class_id, x1, y1, x2, y2 = get_box_info(box, image_width, image_height)
  class_mask_detected = mask[class_id]
  class_mask_scaled = cv2.resize(class_mask_detected, (x2-x1+1, y2-y1+1))
  class_mask_scaled_bool = (class_mask_scaled > mask_threshold)
  
  colorIndex = np.random.randint(0, len(colors)-1)
  color = colors[colorIndex]
  before_masking_roi = draw_image[y1:y2+1, x1:x2+1]
  after_masking_roi = before_masking_roi
  after_masking_roi_bool = after_masking_roi[class_mask_scaled_bool]
  after_masking_roi[class_mask_scaled_bool] = ([0.3 * color[0], 0.3 * color[1], 0.3 * color[2]] + 0.6 * after_masking_roi_bool).astype('uint8')

  class_mask_scaled_int = class_mask_scaled_bool.astype('uint8')
  contours, hierarchy = cv2.findContours(class_mask_scaled_int, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  cv2.drawContours(after_masking_roi, contours, -1, color, 1)
  return draw_image

In [ ]:
def detect_image_mask_rcnn(cv_net, image, confidence_threshold, mask_threshold, use_copied_image=True):
  if use_copied_image:
    draw_image = image.copy()
  else:
    draw_image = image

  image_height = draw_image.shape[0]
  image_width = draw_image.shape[1]
        
  start_time = time.time() 
  cv_net.setInput(cv2.dnn.blobFromImage(draw_image, swapRB=True, crop=False))
  boxes_out, masks_out = cv_net.forward(['detection_out_final', 'detection_masks'])
  inference_time = time.time() - start_time
  print(f'inference time : {np.round(inference_time)}초')

  boxes_detections_nums = boxes_out.shape[2]
  masks_classes_nums = masks_out.shape[1]

  for i in range(boxes_detections_nums):
    box = boxes_out[0, 0, i, :]
    mask = masks_out[i, :, :, :]
    confidence_score = box[2]
    if confidence_score > confidence_threshold:
      draw_box(draw_image, box, image_width, image_height)
      draw_mask(draw_image, box, mask, image_width, image_height, mask_threshold)
  return draw_image

In [ ]:
def detect_video_mask_rcnn(cv_net, input_path, output_path, confidence_threshold, mask_threshold):
  cap = cv2.VideoCapture(video_input_path)
  codec = cv2.VideoWriter_fourcc(*'XVID')
  video_fps = round(cap.get(cv2.CAP_PROP_FPS))
  video_size = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
  frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  video_writer = cv2.VideoWriter(video_output_path, codec, video_fps, video_size)
  print(f'frame count : {frame_count}, video fps : {video_fps}, video size : {video_size}')

  frame_index=0
  while True:
    hasframe, image_frame = cap.read()
    frame_index += 1
    if not hasframe:
      print('process completed')
      break
    print(f'frame index : {frame_index}')
    frame = detect_image_mask_rcnn(cv_net, image_frame, confidence_threshold=confidence_threshold , mask_threshold=mask_threshold, use_copied_image=False)
    video_writer.write(frame)
  
  video_writer.release()
  cap.release()

In [ ]:
video_input_path = './John_Wick_small.mp4'
video_output_path = './drive/MyDrive/Colab Notebooks/update/dl_cv/7_dl_cv_mask_rcnn_inference/data/John_Wick_small_output.avi'

cv_net = cv2.dnn.readNetFromTensorflow('./pretrained/mask_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb', './pretrained/config_mask_graph.pbtxt')
detect_video_mask_rcnn(cv_net, video_input_path, video_output_path, confidence_threshold=0.5, mask_threshold=0.3)

frame count : 58, video fps : 30, video size : (1072, 460)
frame index : 1
inference time : 4.0초
frame index : 2
inference time : 4.0초
frame index : 3
inference time : 4.0초
frame index : 4
inference time : 4.0초
frame index : 5
inference time : 4.0초
frame index : 6
inference time : 4.0초
frame index : 7
inference time : 5.0초
frame index : 8
inference time : 4.0초
frame index : 9
inference time : 4.0초
frame index : 10
inference time : 4.0초
frame index : 11
inference time : 4.0초
frame index : 12
inference time : 4.0초
frame index : 13
inference time : 4.0초
frame index : 14
inference time : 4.0초
frame index : 15
inference time : 4.0초
frame index : 16
inference time : 4.0초
frame index : 17
inference time : 4.0초
frame index : 18
inference time : 4.0초
frame index : 19
inference time : 4.0초
frame index : 20
inference time : 4.0초
frame index : 21
inference time : 4.0초
frame index : 22
inference time : 4.0초
frame index : 23
inference time : 4.0초
frame index : 24
inference time : 4.0초
frame index : 